# **Danish demand and supply 1966-2022**

In this project I will analyze the changes in aggregate demand and aggregate supply in Denmark from 1966 to 2022 by looking at its components. This is done by using data from Statistics Denmark.

The aggregate supply (AS) in a global economy is given by: $AS = Y+M$ 

While the aggregate demand (AD) is: $AD = C+G+I+X$

This gives the total expression in equilibrium: $Y+M=C+G+I+X$

Or alternatively with net exports: $Y=C+G+I+NX$

Where $Y$ is GDP, $M$ is import of goods and services, $C$ is private spending, $G$ is government spending, $I$ is investment, $X$ is export of goods and services, and $NX$ is net exports.

**Imports**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# for import of DST
import requests
from IPython.display import display
from io import StringIO
# for fetching data
import pandas_datareader
from dstapi import DstApi

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# Fetching data

We first need to fetch the data for use in the project. This is done through an API, where the data used is from the dataset 'NAN1' summarized below:

In [2]:
nan1 = DstApi('NAN1')
nan1.tablesummary(language='en')

Table NAN1: Demand and supply by transaction, price unit and time
Last update: 2023-03-31T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,TRANSAKT,31,B1GQK,B.1*g Gross domestic product,EMPM_DC,"Total employment (1,000 persons)",False
1,PRISENHED,6,V_M,"Current prices, (bill. DKK.)",LAN_C,"Pr. capita, 2010-prices, chained values, (1000...",False
2,Tid,57,1966,1966,2022,2022,True


'TRANSAKT' shows the type of transactions there is in the data, while 'PRISENHED' shows what type of prices we can view them in.

In [3]:
#Look at each potential variabel value that 'TRANSAKT' and 'PRISENHED' can take
display(nan1.variable_levels('TRANSAKT',language='en'))
display(nan1.variable_levels('PRISENHED',language='en'))

,id,text
0,B1GQK,B.1*g Gross domestic product
1,P7K,P.7 Imports of goods and services
2,P71K,P.71 Import of goods
3,P72K,P.72 Import of services
4,TFSPR,Supply
5,P6D,P.6 Exports of goods and services
6,P61D,P.61 Export of goods
7,P62D,P.62 Export of services
8,P31S1MD,P.31 Private consumption
9,P31S14D,P.31 Household consumption expenditure


,id,text
0,V_M,"Current prices, (bill. DKK.)"
1,LAN_M,"2010-prices, chained values, (bill. DKK.)"
2,L_V,Period-to-period real growth (per cent)
3,V_C,"Pr. capita. Current prices, (1000 DKK.)"
4,L_VB,"Contribution to GDP growth, (percentage point)"
5,LAN_C,"Pr. capita, 2010-prices, chained values, (1000..."


I select the components for the supply and demand and name them accordingly:

In [4]:
# choose the data to keep
params = {
    'table': 'nan1',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'TRANSAKT', 'values': ['B1GQK','P7K','P6D','P31S1MD','P3S13D','P5GD','TFUPR']},
  {'code': 'PRISENHED', 'values': ['LAN_M']},
  {'code': 'Tid', 'values': ['*']}]}

In [5]:
#rename variable values
nan1_api = nan1.get_data(params=params)
rename = {'B.1*g Gross domestic product':'Y',
          'P.3 Government consumption expenditure':'G',
          'P.31 Private consumption':'C',
          'P.5g Gross capital formation':'I',
          'P.6 Exports of goods and services':'X',
          'P.7 Imports of goods and services':'M',
          'Final demand':'Supply/demand'
}
nan1_api['TRANSAKT']=nan1_api['TRANSAKT'].replace(rename)
#Sort by time and type of transaction
nan1_api.sort_values(by=['TID','TRANSAKT'], inplace=True)
nan1_api.head(7)

,TRANSAKT,PRISENHED,TID,INDHOLD
65,C,"2010-prices, chained values, (bill. DKK.)",1966,372.8
64,G,"2010-prices, chained values, (bill. DKK.)",1966,171.3
66,I,"2010-prices, chained values, (bill. DKK.)",1966,127.6
68,M,"2010-prices, chained values, (bill. DKK.)",1966,112.2
69,Supply/demand,"2010-prices, chained values, (bill. DKK.)",1966,780.4
67,X,"2010-prices, chained values, (bill. DKK.)",1966,123.6
63,Y,"2010-prices, chained values, (bill. DKK.)",1966,702.4


In [6]:
#save the data to ensure we don't lose it
nan1_api.to_csv('nan1_api.csv', index=False)

# Read and clean data

Now that we have the data needed for analysis, we read the data and create a table showing each component value for each year. I also create a variable for the net exports (NX).

In [7]:
#Load the dataset again from the
dkdata = pd.read_csv('nan1_api.csv')
#remove 'PRISENHED' collumn
del dkdata['PRISENHED']
#Create a table
dkbal = dkdata.groupby(['TID', 'TRANSAKT'])['INDHOLD'].mean().unstack(fill_value=0)
#Create net export variable
dkbal['NX']=dkbal.X-dkbal.M
dkbal.tail(5)

TRANSAKT,C,G,I,M,Supply/demand,X,Y,NX
TID,,,,,,,,
2018,975.2,519.2,468.9,1089.6,3178.6,1214.5,2090.4,124.9
2019,990.6,523.6,458.8,1122.4,3242.2,1269.3,2121.6,146.9
2020,977.2,516.4,476.5,1082.5,3160.4,1189.3,2079.3,106.8
2021,1017.7,537.9,505.7,1168.8,3346.0,1284.3,2180.3,115.5
2022,994.5,519.3,566.9,1218.2,3478.5,1394.9,2263.5,176.7


# Analysis

To first get an overview of the data, we view the summary statistics of the components:

In [8]:
dkbal.describe()

TRANSAKT,C,G,I,M,Supply/demand,X,Y,NX
count,57.000000,57.000000,57.00000,57.000000,57.000000,57.000000,57.000000,57.000000
mean,693.078947,373.780702,282.40000,503.252632,1927.449123,584.649123,1441.322807,81.396491
std,181.492019,104.115340,114.33201,335.904705,778.407859,377.845279,436.577260,52.943932
min,372.800000,171.300000,127.60000,112.200000,780.400000,123.600000,702.400000,-7.100000
25%,539.500000,310.200000,183.20000,213.100000,1228.100000,245.300000,1048.100000,33.400000
50%,683.800000,355.500000,236.30000,363.100000,1746.500000,475.100000,1403.300000,96.900000
75%,864.600000,473.400000,359.40000,803.900000,2664.100000,914.900000,1835.100000,121.600000
max,1017.700000,537.900000,566.90000,1218.200000,3478.500000,1394.900000,2263.500000,176.700000


We see a big variance in the import and exports with standard deviations at 336 and 377 respectively. The net exports are for the most part positive with the lowest value being -7B DKK.
Private consumption is the biggest component of GDP while net exports is the lowest.

To further analyze each components impact on GDP, we create a new table showing the ratio between GDP and each component in the early years observed, in the latest years observed and the average over the entire period.

In [9]:
#create a new empty dataframe
GDP_ratio=pd.DataFrame()
components = ['C','G','I','M','X','NX']
#Calculates all GDP ratios over time for each component
for TRANSAKT in components:
    GDP_ratio[f'{TRANSAKT}/Y'] = dkbal[TRANSAKT]/dkbal['Y']*100

print('GDP ratios for 1966-1970 and 2018-2022:')
display(pd.concat([GDP_ratio.head(5),GDP_ratio.tail(5)]))
print('GDP ratio average 1966-2022:')
display(GDP_ratio.mean())

GDP ratios for 1966-1970 and 2018-2022:


,C/Y,G/Y,I/Y,M/Y,X/Y,NX/Y
TID,,,,,,
1966,53.075171,24.387813,18.166287,15.973804,17.596811,1.623007
1967,53.683216,24.959525,18.038316,16.189962,17.296276,1.106314
1968,52.684049,25.012781,18.174847,16.321575,17.983129,1.661554
1969,52.298092,25.129005,19.728789,17.280691,17.904716,0.624025
1970,52.752185,26.257973,19.855894,18.568391,18.296716,-0.271675
2018,46.651359,24.837352,22.431114,52.123995,58.098928,5.974933
2019,46.691176,24.679487,21.625189,52.903469,59.827489,6.924020
2020,46.996585,24.835281,22.916366,52.060790,57.197134,5.136344
2021,46.677063,24.670917,23.194056,53.607302,58.904738,5.297436


GDP ratio average 1966-2022:


C/Y     48.785598
G/Y     26.149671
I/Y     19.119029
M/Y     31.304403
X/Y     36.356394
NX/Y     5.051992
dtype: float64

Here we again see that private spending is the biggest component while government spending and investment follows after.

The ratios for import and export have increased drastically over time going from around 15% in the 60's to over 50% M/Y and 60% X/Y, once again showing the rising trend in foreign trade.

On the opposite side of the spectrum we see that C/Y has gone from 53% in 1966 down to 44% in 2022.

To visualize the developments over time we create an interactive graph:

In [10]:
def plot_e(dkdata, TRANSAKT): 
    I = dkdata['TRANSAKT'] == TRANSAKT
    ax=dkdata.loc[I,:].plot(x='TID', y='INDHOLD', style='-', legend=False, title='Danish supply and demand components 1966-2022, fixed prices: 2010 (Billion DKK)')

In [11]:
# create interactive graph that shows development of each component
widgets.interact(plot_e, 
    dkdata = widgets.fixed(dkdata),
    TRANSAKT = widgets.Dropdown(description='Variable:', 
                                    options=dkdata.TRANSAKT.unique(), 
                                    value='Supply/demand')
);
# display table that shows the component values from 2007-2010
display(dkbal.loc[2007:2010])

interactive(children=(Dropdown(description='Variable:', index=4, options=('C', 'G', 'I', 'M', 'Supply/demand',…

TRANSAKT,C,G,I,M,Supply/demand,X,Y,NX
TID,,,,,,,,
2007,881.6,458.5,439.1,850.8,2730.1,942.6,1879.0,91.8
2008,885.9,473.4,418.0,891.4,2763.3,979.1,1869.4,87.7
2009,855.5,487.8,329.3,784.9,2562.3,888.8,1777.7,103.9
2010,862.2,495.6,327.3,789.1,2600.1,914.9,1810.9,125.8



From the graphs we see a steady increase over time for most components. Investment fluctuates the most and sees a big dip around the financial crisis in the 2009, with investment falling from 418B to 329B. C, M, and X also fall in this period though not as drastic. In total we see a fall of 200B in supply/demand from 2008 to 2009.

Looking at the overall trend in the period we get the table:

In [12]:
print(f'Percentage increase in components 1966-2022:{(dkbal.loc[2022]/dkbal.loc[1966]-1)*100}')

Percentage increase in components 1966-2022:TRANSAKT
C                 166.765021
G                 203.152364
I                 344.278997
M                 985.739750
Supply/demand     345.732957
X                1028.559871
Y                 222.252278
NX               1450.000000
dtype: float64


From the table above we once again see, that trading with foreign countries has increased significantly with import and exports increasing 986% and 1029% respectively, leading to an NX increase of 1450%.

Overall the GDP has incresed 222% while the supply/demand has increased 346%.

# Conclusion

Through the analysis I find, that the danish economy has increased substantially from 1960 to 2022 in all components. The biggest component in GDP over the course of the period is private spending, while import and exports have seen the biggest increase through the period both in terms of the GDP ratio as well as in absolute values.

We see steady growth for most of the periods, though the financial crisis in 2009 had a big short term impact on the economy.